In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import gdown

%matplotlib inline

In [80]:
dom = pd.read_excel('/content/Realestateinvestor_Extract_20230808.xlsx')
lat = pd.read_excel('/content/Latitude_and_Longitude_Extract_20230802.xlsx')

In [81]:
dom['string_from_domain']

0               darwin-city-nt-0800
1                     alawa-nt-0810
2                   brinkin-nt-0810
3             coconut-grove-nt-0810
4                   jingili-nt-0810
                    ...            
12828           gormanston-tas-7466
12829              strahan-tas-7468
12830    granville-harbour-tas-7469
12831         renison-bell-tas-7469
12832               zeehan-tas-7469
Name: string_from_domain, Length: 12833, dtype: object

In [82]:
df = pd.merge(dom, lat, how='left', left_on='string_from_domain',right_on='Search_string')

In [83]:
import pandas as pd
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Conversion factors for radians and Earth's radius
    R = 6371.0  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c

    return distance
# Calculate nearby suburbs and create a new column
max_nearby_suburbs = 40  # Number of nearby suburbs to consider

def find_nearby_suburbs(row):
    if np.isnan(row['lattitude']) or np.isnan(row['longitude']):
        return []  # Return an empty list for NaN values

    distances = haversine(row['lattitude'], row['longitude'], df['lattitude'], df['longitude'])
    sorted_indices = np.argsort(distances)
    nearby_indices = sorted_indices[1 : max_nearby_suburbs + 1]  # Exclude the suburb itself
    nearby_suburb_names = df.loc[nearby_indices, 'string_from_domain'].tolist()
    return nearby_suburb_names

df['nearby_suburbs'] = df.apply(find_nearby_suburbs, axis=1)

# Function to find 3 nearby suburbs with higher prices and their prices
def find_higher_price_suburbs(row):
    central_price = row['median_listing_price_house']
    nearby_suburbs = row['nearby_suburbs']

    higher_price_suburbs = []
    higher_price_prices = []
    for suburb in nearby_suburbs:
        suburb_row = df[df['string_from_domain'] == suburb].iloc[0]
        suburb_price = suburb_row['median_listing_price_house']
        if suburb_price > 1.3 * central_price:
            higher_price_suburbs.append(suburb)
            higher_price_prices.append(suburb_price)
            if len(higher_price_suburbs) == 3:
                break

    return ','.join(higher_price_suburbs), ','.join(map(str, higher_price_prices))

df[['higher_price_suburbs', 'higher_price_prices']] = df.apply(find_higher_price_suburbs, axis=1, result_type='expand')

# ... (continue with the rest of the code)


In [84]:
# df['nearby_suburbs']

In [85]:
df.columns

Index(['postcode', 'state', 'suburb_name', 'string_from_domain',
       'string_from_realestateinvestor', 'population', 'rental_population',
       'current_vacancy_rate', 'rental_stock_available',
       'median_listing_price_house', 'median_listing_price_townhouses',
       'median_listing_price_units', 'median_price_change_last_quarter_house',
       'median_price_change_last_quarter_townhouses',
       'median_price_change_last_quarter_units',
       'median_price_change_1_year_house',
       'median_price_change_1_year_townhouses',
       'median_price_change_1_year_units', 'median_price_change_2_years_house',
       'median_price_change_2_years_townhouses',
       'median_price_change_2_years_units', 'median_weekly_rent_house',
       'median_weekly_rent_townhouses', 'median_weekly_rent_units',
       'median_yield_house', 'median_yield_townhouses', 'median_yield_units',
       'median_rent_change_1_year_house',
       'median_rent_change_1_year_townhouses',
       'median_rent_c

In [86]:
df[['higher_price_suburbs','higher_price_prices','string_from_domain']]

,higher_price_suburbs,higher_price_prices,string_from_domain
0,"larrakeyah-nt-0820,stuart-park-nt-0820,allambi...","1445000.0,920000.0,2150000.0",darwin-city-nt-0800
1,"rapid-creek-nt-0810,nightcliff-nt-0810,muirhea...","725000.0,842500.0,720000.0",alawa-nt-0810
2,,,brinkin-nt-0810
3,,,coconut-grove-nt-0810
4,"nightcliff-nt-0810,ludmilla-nt-0820,allambie-h...","842500.0,780000.0,2150000.0",jingili-nt-0810
...,...,...,...
12828,,,gormanston-tas-7466
12829,"clarence-point-tas-7270,kardinya-wa-6163,point...","740000.0,700000.0,699000.0",strahan-tas-7468
12830,,,granville-harbour-tas-7469
12831,,,renison-bell-tas-7469


In [88]:
df

,postcode,state,suburb_name,string_from_domain,string_from_realestateinvestor,population,rental_population,current_vacancy_rate,rental_stock_available,median_listing_price_house,...,stock_variance_vs_last_year_units,average_days_on_market_house,average_days_on_market_units,Search_string,lat_and_lon,lattitude,longitude,nearby_suburbs,higher_price_suburbs,higher_price_prices
0,800,Northern Territory,Darwin City,darwin-city-nt-0800,darwin-city-nt-0800,6464.0,50.18,1.58,68,490000.0,...,-4.05,106.0,236.0,darwin-city-nt-0800,"-12.4637333,130.8444446",-12.463733,130.844445,"[larrakeyah-nt-0820, stuart-park-nt-0820, alla...","larrakeyah-nt-0820,stuart-park-nt-0820,allambi...","1445000.0,920000.0,2150000.0"
1,810,Northern Territory,Alawa,alawa-nt-0810,alawa-nt-0810,2133.0,27.50,1.25,10,505000.0,...,-14.29,72.0,161.0,alawa-nt-0810,"-12.3811356,130.8752979",-12.381136,130.875298,"[jingili-nt-0810, moil-nt-0810, nakara-nt-0810...","rapid-creek-nt-0810,nightcliff-nt-0810,muirhea...","725000.0,842500.0,720000.0"
2,810,Northern Territory,Brinkin,brinkin-nt-0810,brinkin-nt-0810,1274.0,35.00,1.01,4,NaN,...,33.33,NaN,117.0,brinkin-nt-0810,"-12.3704053,130.8698427",-12.370405,130.869843,"[nakara-nt-0810, tiwi-nt-0810, alawa-nt-0810, ...",,
3,810,Northern Territory,Coconut Grove,coconut-grove-nt-0810,coconut-grove-nt-0810,3057.0,45.85,0.32,5,NaN,...,-10.00,110.0,258.0,coconut-grove-nt-0810,"-12.398889,130.8525",-12.398889,130.852500,"[ludmilla-nt-0820, millner-nt-0810, rapid-cree...",,
4,810,Northern Territory,Jingili,jingili-nt-0810,jingili-nt-0810,1755.0,21.36,0.59,4,589000.0,...,0.00,61.0,NaN,jingili-nt-0810,"-12.3863392,130.8735044",-12.386339,130.873504,"[alawa-nt-0810, moil-nt-0810, millner-nt-0810,...","nightcliff-nt-0810,ludmilla-nt-0820,allambie-h...","842500.0,780000.0,2150000.0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12828,7466,Tasmania,Gormanston,gormanston-tas-7466,gormanston-tas-7466,12.0,NaN,NaN,0,NaN,...,0.00,NaN,NaN,gormanston-tas-7466,"-42.079962,145.6198335",-42.079962,145.619833,"[new-mapoon-qld-4876, beauty-point-tas-7270, e...",,
12829,7468,Tasmania,Strahan,strahan-tas-7468,strahan-tas-7468,705.0,21.88,0.38,2,435000.0,...,100.00,170.0,NaN,strahan-tas-7468,"-42.1828342,145.4170575",-42.182834,145.417057,"[new-mapoon-qld-4876, badger-head-tas-7270, gr...","clarence-point-tas-7270,kardinya-wa-6163,point...","740000.0,700000.0,699000.0"
12830,7469,Tasmania,Granville Harbour,granville-harbour-tas-7469,granville-harbour-tas-7469,28.0,NaN,NaN,0,NaN,...,0.00,NaN,NaN,granville-harbour-tas-7469,"-41.8078417,145.0351942",-41.807842,145.035194,"[greens-beach-tas-7270, flowery-gully-tas-7270...",,
12831,7469,Tasmania,Renison Bell,renison-bell-tas-7469,renison-bell-tas-7469,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,renison-bell-tas-7469,"-41.7952351,145.4358043",-41.795235,145.435804,"[elizabeth-bay-nsw-2011, greens-beach-tas-7270...",,


In [91]:
df['postcode'] = df['postcode'].apply(lambda x: str(x).zfill(4))
del df['nearby_suburbs']

In [92]:
df.to_excel('Near_by_Suburbs_and_Prices.xlsx',index = False)